# Notebook to access to data for the System-level Science Performance Verification Sprint February 3-5 2025

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creattion date : 2025-02-13


- To find what I did on LSSTComCamSim : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/MultiColor_lightCurves-DMRehearsal2024_01-AuxTel-DZPOnCCD.ipynb
- **Confluence page** : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/443613290/Science+Verification+Sprint+Feb+2025


In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = 100000
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
for _ in registry.queryCollections():
    if "LSSTComCam/runs/DRP/DP1/w_2025_05" in _:
        print(_)

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(
        execute=False, exact=False
    ):
        # Limit search results to the data products
        if (
            ("_config" not in datasetType.name)
            and ("_log" not in datasetType.name)
            and ("_metadata" not in datasetType.name)
            and ("_resource_usage" not in datasetType.name)
            and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name) )
        ):
            print(datasetType)

## Select the data product

In [ ]:
FLAG_ISOLATED_STAR_SOURCES = False
FLAG_FGCM = False
FLAG_FGCM_CYCLE5 = True
FLAG_TRANSMISSION = True

In [ ]:
# Try to get the Schema

if FLAG_ISOLATED_STAR_SOURCES:

    all_tracts = []
    data_product = "isolated_star_sources"
    #datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collections, where= where_clause)
    datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection)
    for i, ref in enumerate(datasetRefs):
        print(i,ref.dataId)
        all_tracts.append(ref.dataId["tract"])
        #butler_data = butler.get(ref)

    all_tracts = sorted(all_tracts)
    print("all tracts : ", all_tracts)

#if not isinstance(butler_data, pd.core.frame.DataFrame):
#    print(butler_data.getSchema())


In [ ]:
if FLAG_FGCM_CYCLE5:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection).any(execute=False, exact=False):
            # Limit search results to the data products
            if ("fgcm_Cycle5" in datasetType.name):
                print(datasetType)

In [ ]:
dataid = {'instrument':'LSSTComCam', 'band':'r'}
selected_dataset_type = 'fgcm_Cycle5_Zeropoints_Plot'

In [ ]:
# Does not exists in this collection
#refs =  butler.get(selected_dataset_type,collections=collection,  dataId = dataid)

In [ ]:
set_of_datatypes = ['transmission_optics','transmission_sensor', 'transmission_filter', 'transmission_atmosphere_fgcm' ]

In [ ]:
all_refs = {}
dataid_optics = {'instrument':'LSSTComCam'}
dataid_sensor = {'instrument':'LSSTComCam','detector':0}
dataid_filter = {'band':'r','instrument':'LSSTComCam','physical_filter':'r_03'}
#dataid_filter = {'band':'g','instrument':'LSSTComCam'}


#dataid = {'instrument':'LSSTComCam', 'band':'g','detector':0}
for selected_dataset_type in set_of_datatypes:
    try:
        if selected_dataset_type ==  'transmission_optics':
            therefs = butler.get(selected_dataset_type,collections=collection,  dataId = dataid_optics)
        elif selected_dataset_type ==  'transmission_sensor':
            therefs = butler.get(selected_dataset_type,collections=collection,  dataId = dataid_sensor)
        elif selected_dataset_type ==  'transmission_filter':
            therefs = butler.get(selected_dataset_type,collections=collection,  dataId = dataid_filter)
        else:
            continue
        all_refs[selected_dataset_type] = therefs
    except Exception as inst:
        print(type(inst))    # the exception type

In [ ]:
all_refs

In [ ]:
transmission_optics = all_refs['transmission_optics'] 
transmission_sensor = all_refs['transmission_sensor'] 
transmission_filter = all_refs['transmission_filter'] 

In [ ]:
transmission_optics.getThroughputAtBounds()

In [ ]:
transmission_optics.getWavelengthBounds()

In [ ]:
from lsst.geom import Point2D
wls = np.arange(3200.,10000.,1.)

In [ ]:
transm_optics = transmission_optics.sampleAt(position = Point2D(0.,0.) ,wavelengths = wls)
transm_sensor = transmission_sensor.sampleAt(position = Point2D(0.,0.) ,wavelengths = wls)
transm_filter = transmission_filter.sampleAt(position = Point2D(0.,0.) ,wavelengths = wls)

In [ ]:
plt.plot(wls,transm_optics)
plt.plot(wls,transm_sensor)
plt.plot(wls,transm_filter)